### Importing Libraties

In [2]:
from scipy.optimize import fmin_cg
import pickle

## Policy Network Implementation

In [3]:
# x - vector, that holds the preprocessed pixel information
# W1 and W2 - matrices, that we initialize randomly

h = np.dot(W1, x) # compute hidden layer neuron activations
h[h<0] = 0 # ReLU nonlinearity: threshold at zero
logp = np.dot(W2, h) # compute log probability  of going up
p = 1.0 / (1.0 + np.exp(-logp)) # sigmoid function (gives probability of going up)


NameError: name 'W1' is not defined

### Preprocessing
#### We’ll actually feed subtraction of current and last frame so that it can detect motion

## Credit Assignment Problem

### Policy Gradients

#### We’ll use a 2-layer neural network that takes the raw image pixels (100,800 numbers total (210 * 160 * 3)), and produces a single number indicating the probability of going UP

#### The neurons in the hidden layer (which have their weights arranged along the rows of W1) can detect various game scenarious (e.g. the ball is in the top, and our paddle is in the middle), and the weights in W2 can then decide if in each case we should be going UP or DOWN

#### Our policy network network calculated probability of going UP as 30% (logprob -1.2) and DOWN as 70% (-0.36).
#### We will sample now sample an action from this distribution; E.g. suppose we sample DOWN, and we will execute it in the game. At this point notice one interesting fact: We could immediately fill in a gradient of 1.0 for DOWN as we did in supervised learning, and find the gradient vector that would encourage the network to be slightly more likely to do the DOWN action in the future.
#### So we can immediately evaluate this gradient and that's great, but the problem is that at least for now we do not yet know if going DOWN is good. But the critical point is that that's okay, because we can simply wait a bit and see!
#### For example in Pong we could wait until the end of the game, then take the reward we get and enter that scalar as the gradient for the action we have taken (DOWN in this case).
#### In the example below, going DOWN ended up to us losing the game (-1 reward). So if we fill in -1 for log probability of DOWN and do backprop we will find a gradient that discourages the network to take the DOWN action for that input in the future.

## Training Protocol

#### We will initialize the policy network with some W1, W2 and play 100 games of Pong (we call these policy “rollouts”).
#### Lets assume that each game is made up of 200 frames so in total we’ve made 20,000 decisions for going UP or DOWN, and for each one of these we know the parameter gradient, which tells us how we should change the parameters if we wanted to encourage that decision in that state in the future.
#### All that remains now is to label every decision we've made as good or bad.
#### For example suppose we won 12 games and lost 88. We'll take all 200 * 12 = 2400 decisions we made in the winning games and do a positive update (filling in a +1.0 in the gradient for the sampled action, doing backprop, and parameter update encouraging the actions we picked in all those states.
#### Now we play another 100 games with our new, slightly improved policy and rinse and repeat.

In [3]:
import numpy as np
import gym

In [9]:
decisions = np.array([])

decisions = np.hstack([decisions, 2]) 
decisions = np.hstack([decisions, 2]) 
decisions = np.hstack([decisions, 2]) 
decisions = np.hstack([decisions, 2]) 

np.shape(decisions)

(4,)

In [16]:
env = gym.make('CartPole-v0')

Thetas = np.concatenate([randInitializeWeights(4, 10).flatten(), randInitializeWeights(10, 1).flatten()])


for i_training in range(10):
    X = np.empty((0,4), int)
    decisions = np.array([])
    rewards = np.array([])

    for i_episode in range(10):
        total_reward = 0
        observation = env.reset()
        old_observation = 0
        for t in range(100):
            env.render()
            # call predict() to choose action
            # as an input use substraction from observation the old_observation
            movement = observation-old_observation
            # stack the data in X
            X = np.append(X, np.array([movement]), axis=0)
            prediction = predict(Thetas, movement)
            action = prediction.astype(int)[0][0]
            print(action)
            # stack decisions
            decisions = np.hstack([decisions, action])         

            observation, reward, done, info = env.step(action)

            total_reward = total_reward + reward
            if done:
                print("Episode finished after {} timesteps".format(t+1))
                
                # Stack total_reward t times
                rewards = np.hstack([rewards, np.ones(t+1) * total_reward])
                print("Total reward is {}".format(total_reward))
                break

                
    # select X and decisions with reward more than ...           
    X = X[np.argwhere(rewards>9)]
    decisions = decisions[np.argwhere(rewards>9)]
    
    # shuffling
    #rand_IDX = np.random.permutation(len(decisions))
    #X = X[rand_IDX]
    #decisions = decisions[rand_IDX]

    # training model
    model = fmin_cg_with_thetas(Thetas, X, decisions, 0.01)
    # getting Thetas to use in predictions
    Thetas = model[0]

env.close()

1
1
1
1
1
1
1
1
1
Episode finished after 9 timesteps
Total reward is 9.0
1
1
1
1
1
1
1
1
1
1
Episode finished after 10 timesteps
Total reward is 10.0
1
1
1
1
1
1
1
1
1
Episode finished after 9 timesteps
Total reward is 9.0
1
1
1
1
1
1
1
1
1
1
Episode finished after 10 timesteps
Total reward is 10.0
1
1
1
1
1
1
1
1
1
Episode finished after 9 timesteps
Total reward is 9.0
1
1
1
1
1
1
1
1
1
Episode finished after 9 timesteps
Total reward is 9.0
1
1
1
1
1
1
1
1
1
Episode finished after 9 timesteps
Total reward is 9.0
1
1
1
1
1
1
1
1
1
Episode finished after 9 timesteps
Total reward is 9.0
1
1
1
1
1
1
1
1
Episode finished after 8 timesteps
Total reward is 8.0
1
1
1
1
1
1
1
1
1
1
Episode finished after 10 timesteps
Total reward is 10.0
         Current function value: 0.003188
         Iterations: 5
         Function evaluations: 122
         Gradient evaluations: 110
1
1
1
1
1
1
1
1
1
Episode finished after 9 timesteps
Total reward is 9.0
1
1
1
1
1
1
1
1
1
1
Episode finished after 10 timest

## We just selected the luckiest outcomes from random

In [17]:
env = gym.make('CartPole-v0')
Thetas = np.concatenate([randInitializeWeights(4, 10).flatten(), randInitializeWeights(10, 1).flatten()])
#Thetas = rl_model_hope_3[0]
score = 12
for i_train in range(10):
    
    decisions = []
    X = np.empty((0,4), int)
    rewards = np.array([])
    count = 0
    for i_episode in range(50):
        total_reward = 0
        old_observation = [0,0,0,0]
        observation = env.reset()
        for t in range(100):
            count = count + 1
            env.render()
            
            #print(observation)
            #if t == 0:
            #    predicted_action = predict(Thetas, observation)
            #    #decisions.append(predicted_action)
            #else:
            #    predicted_action = predict(Thetas, X[np.shape(X)[0]-1])
            #    #decisions.append(predicted_action.astype(int)[0][0])
            
            predicted_action = predict(Thetas, observation-old_observation)
            
            old_observation = observation
            
            decisions.append(predicted_action.astype(int)[0][0])    
            #action = env.action_space.sample()
            observation, reward, done, info = env.step(predicted_action.astype(int)[0][0])
            
            print(predicted_action.astype(int))
            print(observation)
            X = np.append(X, np.array([observation-old_observation]), axis=0)
            total_reward = total_reward + reward
            if done:
                rewards = np.hstack([rewards, np.ones(t+1) * total_reward])
                print("Episode finished after {} timesteps".format(t+1))
                print("Total reward is {}".format(total_reward))
                break
    env.close()
    
    #y = np.zeros(count)
   # y[rewards>score] = 1
    #if score < 15:
    #    score = score + 1
    
    #buf = np.array(decisions)
    
    
    
    buf = decisions
    buf = np.array(buf)
    
    
    
    X = X[np.argwhere(rewards>12)]
    buf = buf[np.argwhere(rewards>12)]
    
    #len(buf[np.argwhere(rewards<31)])

    #buf[np.argwhere(rewards<score)] = (buf[np.argwhere(rewards<score)]+1)%2
    #print(buf)
    rl_model = fmin_cg_with_thetas(Thetas, X, buf, 0.1)
    
    Thetas = rl_model[0]

[[0]]
[ 0.01897136 -0.17048237 -0.02470908  0.25840129]
[[0]]
[ 0.01556171 -0.36524301 -0.01954105  0.54318934]
[[0]]
[ 0.00825685 -0.56008496 -0.00867727  0.82965177]
[[0]]
[-0.00294484 -0.75508723  0.00791577  1.11959307]
[[0]]
[-0.01804659 -0.95031212  0.03030763  1.41474845]
[[0]]
[-0.03705283 -1.14579616  0.0586026   1.71674892]
[[0]]
[-0.05996875 -1.3415391   0.09293758  2.0270786 ]
[[0]]
[-0.08679954 -1.53749073  0.13347915  2.34702191]
[[0]]
[-0.11754935 -1.73353504  0.18041959  2.67759906]
[[0]]
[-0.15222005 -1.9294719   0.23397157  3.01948989]
Episode finished after 10 timesteps
Total reward is 10.0
[[0]]
[-0.02875208 -0.17828649  0.04418574  0.35183791]
[[0]]
[-0.03231781 -0.37400803  0.0512225   0.65812005]
[[0]]
[-0.03979797 -0.56980411  0.0643849   0.9664818 ]
[[0]]
[-0.05119405 -0.7657289   0.08371454  1.27867615]
[[0]]
[-0.06650863 -0.96181215  0.10928806  1.59635432]
[[0]]
[-0.08574487 -1.15804701  0.14121515  1.92101728]
[[0]]
[-0.10890581 -1.35437607  0.17963549  2.2

<ipython-input-8-e7694e3f402e>:29: RuntimeWarning: invalid value encountered in true_divide
  Theta1_grad = np.divide(Delta1, m)
<ipython-input-8-e7694e3f402e>:30: RuntimeWarning: invalid value encountered in true_divide
  Theta2_grad = np.divide(Delta2, m)


ZeroDivisionError: float division by zero

In [ ]:
[[1]]
[ 0.15564694  1.77289848 -0.15472481 -2.63595029]
[[1]]
[ 0.19110491  1.96881904 -0.20744382 -2.97162823]
[[1]]
[ 0.23048129  2.16444983 -0.26687638 -3.31933511]
Episode finished after 11 timesteps
Total reward is 11.0
[[0]]
[-0.0036708  -0.20370071 -0.02748916  0.29113948]
[[0]]
[-0.00774482 -0.39842014 -0.02166637  0.57502748]
[[0]]
[-0.01571322 -0.59323176 -0.01016582  0.86080691]


In [56]:
env = gym.make('CartPole-v0')
decisions = []
X = np.empty((0,4), int)
rewards = np.array([])
count = 0
for i_episode in range(2000):
    total_reward = 0
    observation = env.reset()
    for t in range(100):
        old_observation = observation
        count = count + 1
        env.render()
        #print(observation)
        action = env.action_space.sample()
        decisions.append(action)    
        #action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        #print(predicted_action.astype(int)[0][0])
        X = np.append(X, np.array([observation-old_observation]), axis=0)
        total_reward = total_reward + reward
        if done:
            rewards = np.hstack([rewards, np.ones(t+1) * total_reward])
            print("Episode finished after {} timesteps".format(t+1))
            print("Total reward is {}".format(total_reward))
            break
env.close()

Episode finished after 13 timesteps
Total reward is 13.0
Episode finished after 16 timesteps
Total reward is 16.0
Episode finished after 27 timesteps
Total reward is 27.0
Episode finished after 24 timesteps
Total reward is 24.0
Episode finished after 22 timesteps
Total reward is 22.0
Episode finished after 20 timesteps
Total reward is 20.0
Episode finished after 21 timesteps
Total reward is 21.0
Episode finished after 32 timesteps
Total reward is 32.0
Episode finished after 13 timesteps
Total reward is 13.0
Episode finished after 22 timesteps
Total reward is 22.0
Episode finished after 13 timesteps
Total reward is 13.0
Episode finished after 13 timesteps
Total reward is 13.0
Episode finished after 20 timesteps
Total reward is 20.0
Episode finished after 26 timesteps
Total reward is 26.0
Episode finished after 18 timesteps
Total reward is 18.0
Episode finished after 18 timesteps
Total reward is 18.0
Episode finished after 22 timesteps
Total reward is 22.0
Episode finished after 36 times

In [58]:
print(np.mean(rewards))
buf = decisions
buf = np.array(buf)

#len(buf[np.argwhere(rewards<31)])

#buf[np.argwhere(rewards<30)] = (buf[np.argwhere(rewards<30)]+1)%2

X_new = X[np.argwhere(rewards>60)]
buf = buf[np.argwhere(rewards>60)]

print(np.shape(X_new))
print(np.shape(buf))

28.05377313823191
(1819, 1, 4)
(1819, 1)


In [59]:
np.shape(X_new)
decisions_new = decisions
buf_new = buf

In [61]:
rl_model_hope_3 = fmin_cg_train(X_new, buf_new, 0)

Optimization terminated successfully.
         Current function value: 0.000004
         Iterations: 10
         Function evaluations: 47
         Gradient evaluations: 47


In [63]:
predictions = predict(rl_model_hope_3[0], X_new)
predictions
len(predictions[predictions == 1])

array([[0.],
       [0.],
       [1.],
       ...,
       [0.],
       [1.],
       [1.]])

## So our dataset now looks like this:

## Now let's create nn model

### There will be just 4 features in input layer - observation, we'll create one hidden layer with let's say 10 neuron and the output layer will consist of just one neuron - probability of going up

In [4]:
def sigmoid(z):
    #return 1.0 / (1.0 + np.exp(-z))
    return np.divide(1.0, (np.add(1.0, np.exp(np.negative(z)))))

In [5]:
def sigmoidGradient(z):
    return np.multiply(sigmoid(z), np.subtract(1, sigmoid(z)))

In [6]:
def randInitializeWeights(L_in, L_out):
    epsilon_init = np.sqrt(6)/(np.sqrt(L_in + L_out))
    return np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

In [7]:
def costFunction(nn_params, X, y, lmda):
    
    y = np.reshape(y, [len(y),1])
    
    X = np.reshape(X, (-1, 4))
    
    Theta1 = np.reshape(nn_params[0:50], (-1, 5)) 
    Theta2 = np.reshape(nn_params[50:61], (-1, 11))
    
    m = np.shape(X)[0]

    a1 = np.concatenate([np.ones([m, 1]), X], axis=1)

    z2 = np.matmul(a1, np.transpose(Theta1))

    a2 = np.concatenate([np.ones([m, 1]), sigmoid(z2)], axis=1)

    z3 = np.matmul(a2, np.transpose(Theta2))

    a3 = sigmoid(z3)
    
    logErrors = np.multiply(y, np.log(a3)) + np.multiply(np.subtract(1, y), np.log(np.subtract(1, a3)))

    reg = lmda/(2*m) * (np.sum((Theta1[:, 1:]**2)) + np.sum((Theta2[:, 1:]**2)))

    return -1/m * np.sum(logErrors) + reg

In [8]:
def backPropagate(nn_params, X, y, lmda):
    
    y = np.reshape(y, [len(y),1])
    
    X = np.reshape(X, (-1, 4))
    
    Theta1 = np.reshape(nn_params[0:50], (-1, 5)) 
    Theta2 = np.reshape(nn_params[50:61], (-1, 11))
    
    m = np.shape(X)[0]
        
    a1 = np.concatenate([np.ones([m, 1]), X], axis=1)

    z2 = np.matmul(a1, np.transpose(Theta1))

    a2 = np.concatenate([np.ones([m, 1]), sigmoid(z2)], axis=1)

    z3 = np.matmul(a2, np.transpose(Theta2))

    a3 = sigmoid(z3)

    d3 = np.subtract(a3, y)

    d2 = np.multiply(np.matmul(d3, Theta2[:, 1:]), sigmoidGradient(z2))

    Delta1 = np.matmul(np.transpose(d2), a1)
    Delta2 = np.matmul(np.transpose(d3), a2)

    Theta1_grad = np.divide(Delta1, m)
    Theta2_grad = np.divide(Delta2, m)

    Theta1_grad[:, 2:] = Theta1_grad[:, 2:] + lmda/m*Theta1_grad[:, 2:]
    Theta2_grad[:, 2:] = Theta2_grad[:, 2:] + lmda/m*Theta2_grad[:, 2:]

    return np.concatenate([Theta1_grad.flatten(), Theta2_grad.flatten()])

In [9]:
def fmin_cg_train(X_param, y_param, lmda):
    randomThetas_unrolled = np.concatenate([randInitializeWeights(4, 10).flatten(), randInitializeWeights(10, 1).flatten()])
    return fmin_cg(costFunction, fprime=backPropagate, x0=randomThetas_unrolled, args=(X_param.flatten(), y_param.flatten(), lmda), maxiter=1600, disp=True, full_output=True )

In [10]:
def predict(nn_params, X):    
    
    X = np.reshape(X, (-1, 4))
    
    Theta1 = np.reshape(nn_params[0:50], (-1, 5)) 
    Theta2 = np.reshape(nn_params[50:61], (-1, 11))
    
    m = np.shape(X)[0]
        
    a1 = np.concatenate([np.ones([m, 1]), X], axis=1)

    z2 = np.matmul(a1, np.transpose(Theta1))

    a2 = np.concatenate([np.ones([m, 1]), sigmoid(z2)], axis=1)

    z3 = np.matmul(a2, np.transpose(Theta2))

    a3 = sigmoid(z3)
    
    a3[a3>=0.5] = 1
    a3[a3<0.5] = 0
    
    return a3
        

In [11]:
def fmin_cg_with_thetas(Thetas, X_param, y_param, lmda):
    return fmin_cg(costFunction, fprime=backPropagate, x0=Thetas, args=(X_param.flatten(), y_param.flatten(), lmda), maxiter=40, disp=True, full_output=True )

In [12]:
def accuracy(learned_Thetas, X_test, y_test):
    predictions = predict(learned_Thetas, X_test)
    return np.mean((predictions == y_test.flatten()) * 100)